In [14]:
from langchain.tools import BaseTool
import boto3
import json
from pydantic import Field

class BedrockTool(BaseTool):
    """Custom LangChain tool for AWS Bedrock model invocation."""

    name: str = "bedrock_model"
    description: str = "Generates job descriptions using the Claude model via AWS Bedrock."

    client: boto3.client = Field(default_factory=lambda: boto3.client("bedrock-runtime", region_name="us-east-1"))
    model_id: str

    def _run(self, prompt: str) -> str:
        """Invoke the Claude model on AWS Bedrock."""
        try:
            # Prepare the messages for Claude
            messages = [{"role": "user", "content": prompt}]

            # Create the request body
            request_body = json.dumps({
                'messages': messages,
                'max_tokens': 600,
                'temperature': 0.7,
                'anthropic_version': 'bedrock-2023-05-31'
            }).encode('utf-8')

            # Call the model
            response = self.client.invoke_model_with_response_stream(
                modelId=self.model_id,
                body=request_body,
                contentType='application/json',
                accept='application/json'
            )

            # Collect the streamed response
            generated_description = ""
            for event in response["body"]:
                chunk = json.loads(event["chunk"]["bytes"])
                if chunk["type"] == "content_block_delta":
                    generated_description += chunk["delta"].get("text", "")

            return generated_description.strip()

        except Exception as e:
            return f"Error invoking Bedrock model: {str(e)}"

    def _arun(self, *args, **kwargs):
        """Async counterpart of `_run`."""
        raise NotImplementedError("This tool does not support async execution.")


In [10]:
# Define the Bedrock model ID
model_id = 'arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-haiku-20240307-v1:0'

# Initialize the Bedrock tool
bedrock_tool = BedrockTool(model_id=model_id)

In [ ]:
def agent_1_collect_details():
    print("Let's collect the details for the job description.")

    job_title = input("What is the Job Title? ")
    number_of_openings = input("How many openings are there? ")
    location = input("Where is the job located? ")
    salary = input("What is the salary range? ")
    experience_level = input("What is the experience level required? ")
    main_skill = input("What is the main skill required? ")
    company_name = input("What is the company name? ")

    return {
        'job_title': job_title,
        'number_of_openings': number_of_openings,
        'location': location,
        'salary': salary,
        'experience_level': experience_level,
        'main_skill': main_skill,
        'company_name': company_name
    }

# Run Agent 1 to collect job details
collected_details = agent_1_collect_details()


In [ ]:
def agent_2_review_jd(jd_details):
    user_satisfied = False
    attempts = 0
    max_attempts = 3

    # Prepare the prompt for the Claude model
    prompt = (
        f"Generate a concise job description based on the following details:\n"
        f"Job Title: {jd_details['job_title']}\n"
        f"Number of Openings: {jd_details['number_of_openings']}\n"
        f"Location: {jd_details['location']}\n"
        f"Salary: {jd_details['salary']}\n"
        f"Experience Level: {jd_details['experience_level']}\n"
        f"Main Skill: {jd_details['main_skill']}\n"
        f"Company Name: {jd_details['company_name']}\n"
        "Please include job responsibilities and a description."
    )

    while not user_satisfied and attempts < max_attempts:
        # Generate the job description using the Bedrock tool
        generated_description = bedrock_tool._run(prompt)

        if not generated_description:
            return "Error occurred while generating the job description."

        print(f"\nGenerated Job Description:\n{generated_description}\n")

        response = input("Are you satisfied with this job description? (Yes/No): ")

        if response.strip().lower() == 'yes':
            user_satisfied = True
        else:
            print("Let's try generating the details again.")
            attempts += 1

    return generated_description if user_satisfied else "Job description could not be finalized."

# Run Agent 2 to review the job description
final_job_description = agent_2_review_jd(collected_details)

# Output the final result
print("\nFinal Approved Job Description:")
print(final_job_description)
